## Assignment 1

*100 points (8% of course grade)*</br>
*Assigned: Thu, May 22nd*</br>
**Due: Wed, June 4th, 23:59**

This homework should be done in parts as soon as (<= 1 week) relevant topics are covered in lectures. If you wait until the last minute, you might be overwhelmed.

You must turn in the required files electronically, including this Notebook (A1.ipynb) and a few additional files. Please follow the submission instructions for each problem carefully.

In this assignment, you need to solve two problems. In Problem 1, you will write relational algebra queries. In Problem 2, you will draw an E/R diagram.

### Problem 1: Query with Relational Algebra (60%)

Consider a database with following schema: 

- drinker (<u>name</u>, address)
- bar (<u>name</u>, address)
- beer (<u>name</u>, brewer)
- frequents (<u>drinker</u>, <u>bar</u>, times_a_week)
- likes (<u>drinker</u>, <u>beer</u>)
- serves (<u>bar</u>, <u>beer</u>, price)


#### **Preliminary**


To write a relational algebra (RA) query in a cell, we have already converted the cells under each question into the [Markdown cell](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html). Then, you will need to type RA queries in the cell using the syntax in [radb](https://users.cs.duke.edu/~junyang/radb/index.html). Below is an example of an RA query in the radb format. Please refer to [the radb cheatsheet](https://users.cs.duke.edu/~junyang/radb/cheat.html) for the full list of syntax of RA queries. Please use the radb's syntax to ensure we can run your RA queries. 

Example: find the name beers liked by drinkers who frequent the James Joyce Pub bar.
```
\project_{beer} ( 
        (\select_{bar = 'James Joyce Pub'} Frequents)         
         \join_{Frequents.drinker = Likes.drinker}   /* join with Likes to find beers */
         Likes
);
```


You can run RA queries on your local machine after installing radb following the instructions on Canvas. We will also provide an online tool for you to debug your RA queries (stay tuned for the instructions). It will be much easier to grade if we can run your query and also easier for you to debug with query results. The use of the tool is optional, and submissions to the online tool will not be graded (we only grade the Canvas submission).



Now your homework question is to write Relational Algebra queries to answer following questions. 

Please fill your answer in each cell (and **ONLY the query**) and **DO NOT add or remove** any cells to make the TAs' life easier in evaluating your queries. Questions (1)-(6) are worth 6 points each; (7)-(9) are worth 8 points each.


#### 0. (example) Find names of all bars that Eve frequents.

/* input your answer in this cell: */

\project_{bar} (\select_{drinker = 'Eve'} frequents);

#### 1. Find names of beers that  Satisfaction serves

/* input your answer in this cell: */

\project_{beer} (\select_{bar = 'Satisfaction'} serves);


#### 2. Find names of bars that Amy frequents more than once a week

/* input your answer in this cell: */

\project_{name} (\select_{drinker = 'Amy' and times_a_week > 1} frequents);

#### 3. Find names of all drinkers who frequent at least two bars (hint: remember to use renaming)


/* input your answer in this cell: */

\project_{name1} (
    \rename_{name1, bar1, times1} frequents
    \join_{name1 = name2 and bar1 <> bar2}
    \rename_{name2, bar2, times2} frequents
);
/* rename the two frequents tables to self join them */
/* at least two bars -> the same drinker appears in frequents at least twice with different bar names */


#### 4. Find bars frequented by either Ben or Dan, but not both

/* input your answer in this cell: */

\project_{bar} \select_{drinker = 'Ben' or drinker = 'Dan'} frequents
\diff
(\project_{bar} \select_{drinker = 'Ben'} frequents
 \intersect
 \project_{bar} \select_{drinker = 'Dan'} frequents);

#### 5. Find the names of all drinkers who frequent *every* bar (hint: you may store your intermediate results using [views](https://users.cs.duke.edu/~junyang/radb/advance.html?highlight=view) to make your querying process more clear.)

e.g. with view, the first example query in preliminary can be written as

```
v1 :- \select_{bar = 'James Joyce Pub'} Frequents;
\project_{beer} ( 
        v1 
         \join_{v1.drinker = Likes.drinker}   /* join with Likes to find beers */
         Likes
);
```
For view names, please only use **lowercase** letters, because ratest may automatically convert letters to lowercase.

/* input your answer in this cell: */

e1 :- \rename_{bar} \project_{name} bar;

e2 :- \rename_{drinker} \project_{name} drinker;

e3 :- \project_{drinker,bar} (e1 \join e2);
/* the Cartesian product generates all possible combinations of drinkers and bars) */

e4 :- e3 \diff \project_{drinker, bar} frequents;
/* e4 gives all drinkers who do not frequent every bar */
\project_{name} drinker \diff \project_{drinker} e4;


#### 6. Find names and addresses of drinkers who like Corona but do not frequent Satisfaction

/* input your answer in this cell: */

\project_{drinker.name, drinker.address} (
    \select_{beer = 'Corona'} likes \join_{name=drinker} drinker
)
/* gives drinkers who like Corona */

\diff 

\project_{drinker.name, drinker.address} ( 
    \select_{bar = 'Satisfaction'} frequents \join_{name=drinker} drinker
);
/* negates drinkers who frequents Satisfaction*/

#### 7. For each beer that Eve likes, find the names of bars that serve it at the lowest price (when a bar serves multiple beers at the same lowest price, they should all be included in the output) (hint: recall the "trickier exercise" in the slides for how to present "the lowest")

/* input your answer in this cell: */

\project_{beer, bar} (
	(
		\project_{bar, beer} serves
		\diff
		\project_{bar1, beer1} (
		    \rename_{bar1, beer1, price1} serves
			\join_{beer1=beer2 and price1>price2}
		    \rename_{bar2, beer2, price2} serves
	   )
	   /* to find the "lowest": exclude the bar if there's any other bar serves the same beer but with a lower price */
	)
	\join
	\select_{drinker='Eve'} likes
	/* join likes table to obtain the (beer, bar) pairs */
);

#### 8. Find names of all drinkers who frequent *only* those bars that serve *some* beers they like (drinkers who frequent no bars are included)


/* input your answer in this cell: */


// general idea:
// (drinkers who frequent only those bars that serve some beers they like) =
// (drinkers) -
// (drinkers who frequent some bar but like none of the beers served there).

// first, let us find all (drinker, bar) pairs where the bar serves some beer
// that the drinker likes:
e1 :- \project_{drinker, bar} (likes \join serves);

// then, we find all drinkers who frequent some bar that does not serve any
// beer they like:
e2 :- \project_{drinker} (\project_{drinker, bar} frequents \diff e1);

// finally, the answer is given by:
\project_{name} drinker \diff e2;


#### 9. For each beer, find the drinkers who like this beer but frequent *none* of the bars serving this beer. Format your output as a list of (beer, drinker) pairs.

/* input your answer in this cell: */

\project_{likes.beer, likes.drinker} (likes) 
/* get all (beer, drinker) pairs that drinker likes the beer */
\diff
\project_{likes.beer, likes.drinker} (
    likes 
    \join_{likes.drinker = frequents.drinker} frequents 
    \join_{frequents.bar = serves.bar and likes.beer = serves.beer} serves
);
/* negates drinkers who like the beer and frequent some bar serving this beer */

## Problem 2: ER design (40%)

Design a database that captures the following information:
    
    
- Each person is either a student or teacher, but not both.


- Each person has a unique ID, a name, and phone (denoted by the model). 


- The university offers different courses of study. Each course has a unique name and belongs to a department. In any given school year, a given subject can be taught by only one teacher. A course can be taught over multiple years and a student may study the same course multiple times.


- For each student, you need to additionally record the year when he or she entered the university (the class year), as well as his or her favorite subjects.


- A student or a teacher can belong to one or multiple departments. You should be able to track each department's head and its current students.


- Each department has multiple clubs. You should be able to track each club's current students.


Design an E/R diagram for this database. Replace the current `ER-diagram.png` with your figure (you may also use other picture formats like .jpg, but remember to change the filename in the cell below). Very briefly explain the intuitive meaning of any entity and relationship sets as needed. Do not forget to indicate keys and multiplicity of relationships, as well as ISA relationships and weak entity sets (if any), using appropriate notation.


If you think some aspects of the above are unclear, feel free to make additional, reasonable assumptions, but state them clearly in your answer. Also, keep in mind that there is no single “correct” design.

One example design is shown below. Your solution does not have to exactly match the reference design.

<img src="ER-diagram.png" alt="Drawing" style="width: 800px;"/>

`Write a brief explanation for your design in this cell`

### Submission instruction

1. For problem 1, answer the questions (1)-(9) in the Markdown cells.

2. For problem 2, replace `ER-diagram.png` with your ER diagram in a png/jpg file; write some explanation in the Markdown cell.

3. Compress your A1.ipynb (this file) and your ER diagram into A1.zip and submit on Canvas.